### Download Dataset

In [1]:
import kagglehub
import shutil
from pathlib import Path

In [2]:
path = kagglehub.dataset_download("ahmeduzaki/global-earthquake-tsunami-risk-assessment-dataset")

In [3]:
raw_dir = Path("../data/raw")
download_dir = Path(path)
if not download_dir.exists():
    print(f"Download directory {download_dir} does not exist, skipping.")
dataset_name = download_dir.name
dataset_raw_dir = raw_dir / dataset_name
dataset_raw_dir.mkdir(parents=True, exist_ok=True)
for item in download_dir.iterdir():
    if item.is_file():
        dest = dataset_raw_dir / item.name
        shutil.copy2(item, dest)

### Download Tsunami Dataset

In [4]:
path_tsunami = kagglehub.dataset_download("andrewmvd/tsunami-dataset")

In [5]:
download_dir = Path(path_tsunami)
if not download_dir.exists():
    print(f"Download directory {download_dir} does not exist, skipping.")
tsunami_dataset_name = download_dir.name
tsunami_dataset_raw_dir = raw_dir / tsunami_dataset_name
tsunami_dataset_raw_dir.mkdir(parents=True, exist_ok=True)
for item in download_dir.iterdir():
    if item.is_file():
        dest = tsunami_dataset_raw_dir / item.name
        shutil.copy2(item, dest)
        
